<a href="https://colab.research.google.com/github/mhrgroup/course_self_supervised_learning/blob/main/Section%2002%3A%20Supervised%20Models/ssl_section02_lecture03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture 03: Supervised Learning**

By the end of this lecture, you will be able to:

1. Describe supervised learning in general.
2. Implement supervised learning with randomly generated parameters.

# **3.1. Supervised Learning in General**
---

* Supervised learning is about learning from labeled data or data from outputs.
* A machine learning model is a classifier designed to learn from input and categorical output data (classification problem).
* When outputs are real-values, it is called a regressor (regression problem).
* Machine learning models solely learn from input data are often called unsupervised machine learning models.
* In this course, the machine learning models are all multi-layer deep neural network classifiers with layers including but not limited to convolutions, pooling, recurrent, and dense layers. They are since referred to as “models.”

# **3.2. Supervised Learning with Randomly Generated Parameters**
---
* One can develop a supervised model with randomly initiated parameters.
* Next, using a learning technique, such as Adam or Stochastic Gradient Descent (SGD), the parameters are learned such the model accurately maps the outputs.
* We will see an example of a supervised classification model in this lecture section using the CIFAR-10 dataset.
* CIFAR-10 dataset contains 60,000 32x32 color images in ten classes: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks (6,000 images of each class).
* CIFAR-10 is divided into 50,000 training and 10,000 testing data points (i.e., images). For more, navigate to https://en.wikipedia.org/wiki/CIFAR-10.

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/cifar10.png	"	width="650"/>

> **Abbreviations:**
*	datain: input data
*	dataou: output data
*	te: testing
*	tf: tensorflow
*	tr: training

In [ ]:
#@title Install necessary libraries & restart the session

# Install the required libraries using the `pip` package manager.
!pip install tensorflow==2.15

# Import the time module to add a delay before restarting the session.
import time

# Import `clear_output` from IPython to clear the notebook output, ensuring a clean display for the user.
from IPython.display import clear_output

# Clear the output after the packages are installed to make the notebook cleaner.
clear_output()

# Print a message to let the user know that the libraries are installed & the session will restart.
print("Necessary Libraries are Installed. Restarting the session!")

# Add a short delay (1 second) before restarting to allow the message to be displayed to the user.
time.sleep(1)

# Import the `os` module to access low-level operating system functionality.
import os

# Use `os._exit(00)` to exit the current Python runtime environment forcefully.
# This effectively simulates a restart in notebook environments like Google Colab or Jupyter.
# After this command, the environment will be restarted & all the packages installed will be properly loaded.
os._exit(00)

In [ ]:
#@title Import necessary libraries
import time
import tensorflow as tf

In [ ]:
#@title Load and process the CIFAR-10 data
(datain_tr, dataou_tr), (datain_te, dataou_te) = tf.keras.datasets.cifar10.load_data()

'''
List of available datasets at tf.keras.datasets:
https://www.tensorflow.org/api_docs/python/tf/keras/datasets.
'''

datain_tr = datain_tr/255 # trasnform unit-8 values between 0 and 1
datain_te = datain_te/255 # trasnform unit-8 values between 0 and 1

dataou_tr = tf.keras.utils.to_categorical(dataou_tr) # e.g. class 3 out of 4 becomes [0,0,1,0]
dataou_te = tf.keras.utils.to_categorical(dataou_te)

# print shapes of data
print('Shape of datain_tr: {}'.format(datain_tr.shape))
print('Shape of datain_te: {}'.format(datain_te.shape))
print('Shape of dataou_tr: {}'.format(dataou_tr.shape))
print('Shape of dataou_te: {}'.format(dataou_te.shape))


In [ ]:
#@title Create a model similar to DenseNet121
'''
DenseNet121 is a deep neural network tower including convolutional, pooling,
and dense layers. You can learn about it at https://arxiv.org/abs/1608.06993.
'''

layerin = tf.keras.Input(shape=(datain_tr.shape[1],
                                datain_tr.shape[2],
                                datain_tr.shape[3]))

'''
The DenseNet121 input size is set to 160 by 160 by 3, and our images are
32 by 32 by 3. The upscale is to resize our 32 by 32 images to become 160 by 160
for compatibility.

Read about tf.keras.layers.Lambda here:
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda.
'''

upscale = tf.keras.layers.Lambda(lambda x: tf.image.resize_with_pad(x,
                                                                    160,
                                                                    160,
                                                                    method = tf.image.ResizeMethod.BILINEAR))(layerin)
'''
Read more about image ResizeMethod at
https://www.tensorflow.org/api_docs/python/tf/image/resize.
'''



model_DenseNet121 = tf.keras.applications.DenseNet121(include_top  = False,
                                                      weights      = None,
                                                      input_shape  = (160,160,3),
                                                      input_tensor = upscale,
                                                      pooling      = 'max')

'''
read more at
https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet/DenseNet121.
'''

layerou = tf.keras.layers.Dense(dataou_tr.shape[-1], activation = 'softmax')

'''
Create the model!
We add "BatchNormalization" to reduce overfitting.
A useful article to learn more about BacthNormalization:
https://towardsdatascience.com/batch-norm-explained-visually-how-it-works-and-why-neural-networks-need-it-b18919692739.
'''

model   = tf.keras.models.Sequential([model_DenseNet121,
                                      tf.keras.layers.BatchNormalization(),
                                      layerou])


model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss      = 'categorical_crossentropy',
              metrics   = ['accuracy'])

'''
Print model architectures.
'''
print('summary of model_DenseNet121:\n')
model_DenseNet121.summary()

print('summary of model:\n')
model.summary()

In [ ]:
#@title Train the model for five epochs
t_start = time.time()

history = model.fit(datain_tr,
                    dataou_tr,
                    epochs           = 5,
                    batch_size       = 128,
                    verbose          = 1,
                    shuffle          = True,
                    validation_split = 0.05)

t_end   = time.time()



Epoch 1/5
372/372 [==============================] - 354s 730ms/step - loss: 1.4469 - accuracy: 0.4928 - val_loss: 1.9681 - val_accuracy: 0.3284
Epoch 2/5
372/372 [==============================] - 262s 705ms/step - loss: 1.0115 - accuracy: 0.6563 - val_loss: 1.9623 - val_accuracy: 0.3908
Epoch 3/5
 46/372 [==>...........................] - ETA: 3:47 - loss: 0.8978 - accuracy: 0.6856

In [ ]:
#@title Compute testing accuracy
_, accuracy_te = model.evaluate(datain_te,
                                dataou_te,
                                batch_size = 128)

print('\nTraining time: {:06.2f} sec'.format(t_end - t_start))
print('\nTesting acuuracy: {:05.2f}%'.format(accuracy_te * 100))

In [ ]:
#@title Clean up memory
%reset

# **Lecture 03: Supervised Learning**

In this lecture, you learned about:

1. Supervised learning in general.
2. Supervised learning with randomly generated parameters.

> ***In the following lecture, we will talk about "Transfer Learning & Fine-Tuning."***